In [1]:
import numpy as np
import pandas as pd
from ast import literal_eval

In [2]:
# Import BVS_result
# Metal charge by cell2mol or BVS (only cases that cell reconstruction finished successfully)
df_bvs = pd.read_csv("BVS_result_Cu.txt",delimiter="\t")

In [3]:
df_bvs # 2493 Complexes

,refcode,metal,charge_cell2mol,charge_BVS,dictionary,unique_indices
0,ABAFAN,Cu,1,7777,"{1: 0.00199, 2: 0.015306, 3: 0.988304}",2
1,ABAFER,Cu,1,7777,"{1: 0.001237, 2: 0.132774, 3: 1.10737}",2
2,ABAMUO,Cu,2,7777,"{1: 0.284689, 2: 1.9e-05, 3: 0.482799}",3
3,ABETEH,Cu,1,7777,"{1: 0.002921, 2: 0.001846, 3: 9999}",2
4,ABETIL,Cu,1,7777,"{1: 0.019641, 2: 0.000227, 3: 9999}",2
...,...,...,...,...,...,...
11356,ZUYWAU,Cu,9999,7777,"{1: 0.256987, 2: 6.9e-05, 3: 0.443784}",2
11357,ZUZKAG,Cu,2,7777,"{1: 0.206968, 2: 7.6e-05, 3: 0.461207}",2
11358,ZUZKOX,Cu,1,9999,"{1: 9999, 2: 9999, 3: 9999}",2
11359,ZUZZEC,Cu,2,7777,"{1: 0.236939, 2: 8.2e-05, 3: 0.492313}",1


In [4]:
df_cif = pd.read_csv("cif_charge_Cu.txt",delimiter="\t") # charge extracted from cif files

In [5]:
df_cif # 2831 complexes

,refcode,metal,charge_cif
0,ABAFAN,Cu,1
1,ABAFER,Cu,1
2,ABAMUO,Cu,2
3,ABAXEI,Cu,2
4,ABAXIM,Cu,2
...,...,...,...
12858,ZUZKAG,Cu,2
12859,ZUZKOX,Cu,1
12860,ZUZNIR,Cu,2
12861,ZUZZEC,Cu,2


In [6]:
df = df_bvs.merge(df_cif, on='refcode', how='left')
df = df.drop(['metal_y', 'unique_indices'], axis=1)
df = df.rename(columns={'metal_x': 'metal'})
df['dictionary'] = df.dictionary.apply(lambda x: literal_eval(str(x)))
cols = ['refcode', 'metal', 'charge_cif', 'charge_cell2mol', 'charge_BVS', 'dictionary']
df = df[cols] 
df = df.rename(columns={'charge_cif': 'Cif', 'charge_cell2mol' :'cell2mol', 'charge_BVS' : 'BVS'})

In [7]:
thre_max = 0.5

In [8]:
df['bvs_min'] = df.dictionary.apply(lambda x: sorted(x, key=x.get)[0]
                                      if x[sorted(x, key=x.get)[0]] <= thre_max else False)
df['min'] = df.dictionary.apply(lambda x: x[sorted(x, key=x.get)[0]]
                                      if x[sorted(x, key=x.get)[0]] <= thre_max else False)
df['bvs_second_min'] = df.dictionary.apply(lambda x: sorted(x, key=x.get)[1]
                                      if x[sorted(x, key=x.get)[1]] <= thre_max else False)
df['second_min'] = df.dictionary.apply(lambda x: x[sorted(x, key=x.get)[1]]
                                      if x[sorted(x, key=x.get)[1]] <= thre_max else False)

In [9]:
condition = (lambda x: 
 x[sorted(x, key=x.get)[1]]- x[sorted(x, key=x.get)[0]]  # (second_min_delta - min_delta)
 if (x[sorted(x, key=x.get)[0]] <= thre_max and x[sorted(x, key=x.get)[1]] <= thre_max) 
 else False)
df['diff'] = df.dictionary.apply(condition)

In [10]:
df

,refcode,metal,Cif,cell2mol,BVS,dictionary,bvs_min,min,bvs_second_min,second_min,diff
0,ABAFAN,Cu,1,1,7777,"{1: 0.00199, 2: 0.015306, 3: 0.988304}",1,0.00199,2,0.015306,0.013316
1,ABAFER,Cu,1,1,7777,"{1: 0.001237, 2: 0.132774, 3: 1.10737}",1,0.001237,2,0.132774,0.131537
2,ABAMUO,Cu,2,2,7777,"{1: 0.284689, 2: 1.9e-05, 3: 0.482799}",2,0.000019,1,0.284689,0.28467
3,ABETEH,Cu,1,1,7777,"{1: 0.002921, 2: 0.001846, 3: 9999}",2,0.001846,1,0.002921,0.001075
4,ABETIL,Cu,1,1,7777,"{1: 0.019641, 2: 0.000227, 3: 9999}",2,0.000227,1,0.019641,0.019414
...,...,...,...,...,...,...,...,...,...,...,...
11356,ZUYWAU,Cu,2,9999,7777,"{1: 0.256987, 2: 6.9e-05, 3: 0.443784}",2,0.000069,1,0.256987,0.256918
11357,ZUZKAG,Cu,2,2,7777,"{1: 0.206968, 2: 7.6e-05, 3: 0.461207}",2,0.000076,1,0.206968,0.206892
11358,ZUZKOX,Cu,1,1,9999,"{1: 9999, 2: 9999, 3: 9999}",False,False,False,False,False
11359,ZUZZEC,Cu,2,2,7777,"{1: 0.236939, 2: 8.2e-05, 3: 0.492313}",2,0.000082,1,0.236939,0.236857


In [ ]:
df[(df['cell2mol'] != 9999) & (df['cell2mol'] != df['Cif'])]

In [54]:
df[(df['bvs_min'] != False) & (df['cell2mol'] != df['Cif'])]
# df[(df['bvs_min'] == False) ]

,refcode,metal,Cif,cell2mol,BVS,dictionary,bvs_min,min,bvs_second_min,second_min,diff,_0.1,_0.2,_0.3,_0.4,_0.5
18,ABPYCU,Cu,2,9999,7777,"{1: 0.132927, 2: 6e-06, 3: 0.532555}",2,0.000006,1,0.132927,0.132921,2,False,False,False,False
43,ACMCUB,Cu,2,9999,7777,"{1: 0.164634, 2: 6e-05, 3: 9999}",2,0.00006,1,0.164634,0.164574,2,False,False,False,False
47,ACOXNI10,Cu,2,9999,7777,"{1: 0.146472, 2: 1.1e-05, 3: 0.503055}",2,0.000011,1,0.146472,0.146461,2,False,False,False,False
54,ACXACU,Cu,2,9999,7777,"{1: 0.207636, 2: 0.002269, 3: 0.654437}",2,0.002269,1,0.207636,0.205367,2,2,False,False,False
69,AEPCUI,Cu,2,9999,7777,"{1: 0.002664, 2: 0.014632, 3: 0.987621}",1,0.002664,2,0.014632,0.011968,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11352,ZUYTUL,Cu,2,9999,7777,"{1: 0.254451, 2: 8.1e-05, 3: 0.471568}",2,0.000081,1,0.254451,0.25437,2,2,False,False,False
11353,ZUYVAT,Cu,2,9999,7777,"{1: 0.324099, 2: 0.000676, 3: 0.406939}",2,0.000676,1,0.324099,0.323423,2,2,2,False,False
11354,ZUYVIB,Cu,2,9999,7777,"{1: 0.267311, 2: 0.000353, 3: 0.451893}",2,0.000353,1,0.267311,0.266958,2,2,False,False,False
11355,ZUYVUN,Cu,2,9999,7777,"{1: 0.255744, 2: 0.000103, 3: 0.469588}",2,0.000103,1,0.255744,0.255641,2,2,False,False,False


In [16]:
print("Total cases : {}".format(len(df)))
print("Cell2mol and Cif are same : {} cases / different {} cases".format(
    len(df[df['cell2mol'] == df['Cif']]), len(df[df['cell2mol'] != df['Cif']])))
print("bvs_min and Cif are same : {} cases / different {} cases".format(
    len(df[df['bvs_min'] == df['Cif']]), len(df[df['bvs_min'] != df['Cif']])))
df[['refcode', 'Cif', 'cell2mol', 'bvs_min', 'BVS']]

Total cases : 11361
Cell2mol and Cif are same : 9791 cases / different 1570 cases
bvs_min and Cif are same : 10417 cases / different 944 cases


,refcode,Cif,cell2mol,bvs_min,BVS
0,ABAFAN,1,1,1,7777
1,ABAFER,1,1,1,7777
2,ABAMUO,2,2,2,7777
3,ABETEH,1,1,2,7777
4,ABETIL,1,1,2,7777
...,...,...,...,...,...
11356,ZUYWAU,2,9999,2,7777
11357,ZUZKAG,2,2,2,7777
11358,ZUZKOX,1,1,False,9999
11359,ZUZZEC,2,2,2,7777


In [26]:
df['_0.1'] = df.apply((lambda x: x['bvs_min'] if (x['diff'] > 0.1 and x['diff'] != False) else False), axis=1)
df['_0.2'] = df.apply((lambda x: x['bvs_min'] if (x['diff'] > 0.2 and x['diff'] != False) else False), axis=1)
df['_0.3'] = df.apply((lambda x: x['bvs_min'] if (x['diff'] > 0.3 and x['diff'] != False) else False), axis=1)
df['_0.4'] = df.apply((lambda x: x['bvs_min'] if (x['diff'] > 0.4 and x['diff'] != False) else False), axis=1)
df['_0.5'] = df.apply((lambda x: x['bvs_min'] if (x['diff'] > 0.5 and x['diff'] != False) else False), axis=1)

In [27]:
df

,refcode,metal,Cif,cell2mol,BVS,dictionary,bvs_min,min,bvs_second_min,second_min,diff,_0.1,_0.2,_0.3,_0.4,_0.5
0,ABAFAN,Cu,1,1,7777,"{1: 0.00199, 2: 0.015306, 3: 0.988304}",1,0.00199,2,0.015306,0.013316,False,False,False,False,False
1,ABAFER,Cu,1,1,7777,"{1: 0.001237, 2: 0.132774, 3: 1.10737}",1,0.001237,2,0.132774,0.131537,1,False,False,False,False
2,ABAMUO,Cu,2,2,7777,"{1: 0.284689, 2: 1.9e-05, 3: 0.482799}",2,0.000019,1,0.284689,0.28467,2,2,False,False,False
3,ABETEH,Cu,1,1,7777,"{1: 0.002921, 2: 0.001846, 3: 9999}",2,0.001846,1,0.002921,0.001075,False,False,False,False,False
4,ABETIL,Cu,1,1,7777,"{1: 0.019641, 2: 0.000227, 3: 9999}",2,0.000227,1,0.019641,0.019414,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11356,ZUYWAU,Cu,2,9999,7777,"{1: 0.256987, 2: 6.9e-05, 3: 0.443784}",2,0.000069,1,0.256987,0.256918,2,2,False,False,False
11357,ZUZKAG,Cu,2,2,7777,"{1: 0.206968, 2: 7.6e-05, 3: 0.461207}",2,0.000076,1,0.206968,0.206892,2,2,False,False,False
11358,ZUZKOX,Cu,1,1,9999,"{1: 9999, 2: 9999, 3: 9999}",False,False,False,False,False,False,False,False,False,False
11359,ZUZZEC,Cu,2,2,7777,"{1: 0.236939, 2: 8.2e-05, 3: 0.492313}",2,0.000082,1,0.236939,0.236857,2,2,False,False,False


In [28]:
df[(df['BVS'] == 7777) & (df['diff'] > 0.1)]
df[(df['BVS'] == 7777) & (df['diff'] > 0.2)]
df[(df['BVS'] == 7777) & (df['diff'] > 0.3)]
df[(df['BVS'] == 7777) & (df['diff'] > 0.4)]

,refcode,metal,Cif,cell2mol,BVS,dictionary,bvs_min,min,bvs_second_min,second_min,diff,_0.1,_0.2,_0.3,_0.4,_0.5
84,AFEZAP,Cu,2,9999,7777,"{1: 0.469075, 2: 0.056463, 3: 9999}",2,0.056463,1,0.469075,0.412612,2,2,2,2,False
202,ALIPER,Cu,1,1,7777,"{1: 0.000744, 2: 0.440566, 3: 9999}",1,0.000744,2,0.440566,0.439822,1,1,1,1,False
275,APUYOB,Cu,1,1,7777,"{1: 0.034367, 2: 0.487977, 3: 9999}",1,0.034367,2,0.487977,0.45361,1,1,1,1,False
387,AVIFAO,Cu,1,1,7777,"{1: 0.059699, 2: 0.460711, 3: 1.215591}",1,0.059699,2,0.460711,0.401012,1,1,1,1,False
396,AVODIZ,Cu,1,1,7777,"{1: 0.006865, 2: 0.458305, 3: 9999}",1,0.006865,2,0.458305,0.45144,1,1,1,1,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11004,YUGLOB,Cu,1,1,7777,"{1: 0.012482, 2: 0.496109, 3: 1.475648}",1,0.012482,2,0.496109,0.483627,1,1,1,1,False
11047,YUYBUP,Cu,2,2,7777,"{1: 0.003133, 2: 0.447797, 3: 1.322066}",1,0.003133,2,0.447797,0.444664,1,1,1,1,False
11056,ZACGIT,Cu,1,1,7777,"{1: 0.00779, 2: 0.488828, 3: 1.319826}",1,0.00779,2,0.488828,0.481038,1,1,1,1,False
11057,ZACGOZ,Cu,1,1,7777,"{1: 0.000271, 2: 0.400454, 3: 1.221743}",1,0.000271,2,0.400454,0.400183,1,1,1,1,False


In [29]:
df_diff = df[df['diff'] != False]

In [30]:
df_diff = df_diff.drop(['metal', 'BVS', 'dictionary', 'min', 'second_min'], axis=1)

In [31]:
df_diff

,refcode,Cif,cell2mol,bvs_min,bvs_second_min,diff,_0.1,_0.2,_0.3,_0.4,_0.5
0,ABAFAN,1,1,1,2,0.013316,False,False,False,False,False
1,ABAFER,1,1,1,2,0.131537,1,False,False,False,False
2,ABAMUO,2,2,2,1,0.28467,2,2,False,False,False
3,ABETEH,1,1,2,1,0.001075,False,False,False,False,False
4,ABETIL,1,1,2,1,0.019414,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...
11355,ZUYVUN,2,9999,2,1,0.255641,2,2,False,False,False
11356,ZUYWAU,2,9999,2,1,0.256918,2,2,False,False,False
11357,ZUZKAG,2,2,2,1,0.206892,2,2,False,False,False
11359,ZUZZEC,2,2,2,1,0.236857,2,2,False,False,False


In [32]:
for i in [0.1, 0.2, 0.3, 0.4, 0.5]:
    thre_diff = i
    # len(df_diff[df_diff[f'_{thre_diff}'] != False])
    print(f"The minimum diffrence between smallest delta and second smallest delat is {thre_diff}")
    print("The number of cases that bvs doesn't work in the above condition is {} out of {}\n".format(
        len(df_diff[(df_diff[f'_{thre_diff}'] != df_diff['Cif'])& (df_diff[f'_{thre_diff}'] != False)]), len(df_diff[df_diff[f'_{thre_diff}'] != False])))

The minimum diffrence between smallest delta and second smallest delat is 0.1
The number of cases that bvs doesn't work in the above condition is 209 out of 7960

The minimum diffrence between smallest delta and second smallest delat is 0.2
The number of cases that bvs doesn't work in the above condition is 79 out of 4111

The minimum diffrence between smallest delta and second smallest delat is 0.3
The number of cases that bvs doesn't work in the above condition is 46 out of 973

The minimum diffrence between smallest delta and second smallest delat is 0.4
The number of cases that bvs doesn't work in the above condition is 23 out of 115

The minimum diffrence between smallest delta and second smallest delat is 0.5
The number of cases that bvs doesn't work in the above condition is 0 out of 0



In [43]:
df_diff[df_diff['diff']< 0.1]

,refcode,Cif,cell2mol,bvs_min,bvs_second_min,diff,_0.1,_0.2,_0.3,_0.4,_0.5
0,ABAFAN,1,1,1,2,0.013316,False,False,False,False,False
3,ABETEH,1,1,2,1,0.001075,False,False,False,False,False
4,ABETIL,1,1,2,1,0.019414,False,False,False,False,False
9,ABODUS,1,1,1,2,0.043813,False,False,False,False,False
10,ABOFAA,1,1,1,2,0.03856,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...
11341,ZURNUV,1,1,1,2,0.056949,False,False,False,False,False
11343,ZURZIV,2,2,2,1,0.000028,False,False,False,False,False
11344,ZURZOB,2,2,2,1,0.009774,False,False,False,False,False
11346,ZUWLUY,2,2,2,1,0.055877,False,False,False,False,False


In [33]:
thre_diff = 0.1
print(len(df_diff[(df_diff[f'_{thre_diff}'] != df_diff['Cif'])& (df_diff[f'_{thre_diff}'] != False)]))
df_diff[(df_diff[f'_{thre_diff}'] != df_diff['Cif'])& (df_diff[f'_{thre_diff}'] != False)]

209


,refcode,Cif,cell2mol,bvs_min,bvs_second_min,diff,_0.1,_0.2,_0.3,_0.4,_0.5
88,AFOLOW,2,2,3,2,0.110253,3,False,False,False,False
177,AKIYUO,3,2,2,1,0.257059,2,2,False,False,False
251,ANICEI,3,1,2,3,0.184052,2,False,False,False,False
264,ANUNIH,2,2,1,2,0.131926,1,False,False,False,False
305,ARAVOE,2,2,1,2,0.137523,1,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...
11087,ZANDAT,1,1,2,1,0.117844,2,False,False,False,False
11128,ZEBRUT,2,2,3,2,0.356913,3,3,3,False,False
11165,ZETDOR,2,2,1,2,0.102331,1,False,False,False,False
11193,ZIDBIA,3,9999,2,1,0.413458,2,2,2,2,False


In [27]:
thre_diff = 0.2
print(len(df_diff[(df_diff[f'_{thre_diff}'] != df_diff['Cif'])& (df_diff[f'_{thre_diff}'] != False)]))
df_diff[(df_diff[f'_{thre_diff}'] != df_diff['Cif'])& (df_diff[f'_{thre_diff}'] != False)]

8


,refcode,Cif,cell2mol,bvs_min,bvs_second_min,diff,_0.1,_0.2,_0.3,_0.4,_0.5
181,BOVGID,2,1,3,2,0.378022,3,3,3,False,False
1053,LEHSIB,2,2,3,2,0.350357,3,3,3,False,False
1186,MNICUR,2,9999,3,2,0.448291,3,3,3,3,False
1395,OMOGEC,4,9999,2,3,0.259465,2,2,False,False,False
1769,SIQMIO,5,9999,2,3,0.234226,2,2,False,False,False
2006,USIFOU,2,3,3,2,0.36927,3,3,3,False,False
2279,XILLIO,2,2,3,4,0.378837,3,3,3,False,False
2467,ZOFFEI,3,1,4,3,0.208904,4,4,False,False,False


In [28]:
thre_diff = 0.3
print(len(df_diff[(df_diff[f'_{thre_diff}'] != df_diff['Cif'])& (df_diff[f'_{thre_diff}'] != False)]))
df_diff[(df_diff[f'_{thre_diff}'] != df_diff['Cif'])& (df_diff[f'_{thre_diff}'] != False)]

5


,refcode,Cif,cell2mol,bvs_min,bvs_second_min,diff,_0.1,_0.2,_0.3,_0.4,_0.5
181,BOVGID,2,1,3,2,0.378022,3,3,3,False,False
1053,LEHSIB,2,2,3,2,0.350357,3,3,3,False,False
1186,MNICUR,2,9999,3,2,0.448291,3,3,3,3,False
2006,USIFOU,2,3,3,2,0.36927,3,3,3,False,False
2279,XILLIO,2,2,3,4,0.378837,3,3,3,False,False


In [29]:
thre_diff = 0.4
print(len(df_diff[(df_diff[f'_{thre_diff}'] != df_diff['Cif'])& (df_diff[f'_{thre_diff}'] != False)]))
df_diff[(df_diff[f'_{thre_diff}'] != df_diff['Cif'])& (df_diff[f'_{thre_diff}'] != False)]

1


,refcode,Cif,cell2mol,bvs_min,bvs_second_min,diff,_0.1,_0.2,_0.3,_0.4,_0.5
1186,MNICUR,2,9999,3,2,0.448291,3,3,3,3,False
